In [127]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
data = pd.read_csv('DEAM_CSMT/arousal-valence.csv')#读取CSV文件并将第一列song_id作为index
data.head()

,index,genre,coefficient,R2,pacf_max,pacf_index,pacf_abs_rate,pacf_rel_rate,if_peroid,pvalue,entropy,coefficient.1,R2.1,pacf_max.1,pacf_index.1,pacf_abs_rate.1,pacf_rel_rate.1,if_peroid.1,pvalue.1,entropy.1
0,0,blues,-0.003153,0.841198,4.143352,20,0.165371,1.061999,1,0.345547,5.211211,-0.005579,0.914857,0.433067,26,0.135172,1.264316,1,0.079487,5.072140
1,1,blues,-0.002307,0.881956,7.572991,20,0.241182,1.015799,1,0.537526,5.211211,-0.002212,0.709225,4.376909,29,0.457470,3.987790,1,0.000417,4.937500
2,2,blues,0.002948,0.833641,1.818423,28,0.246616,1.199912,1,0.088967,5.044394,0.001939,0.527923,17.170336,28,0.745046,16.517620,1,0.479829,5.044394
3,3,blues,0.000424,0.611978,33.036178,29,0.875388,32.149353,1,0.022661,4.663910,-0.000481,0.144355,0.307684,5,0.104818,1.064716,1,0.000016,4.972446
4,4,blues,-0.002731,0.775485,0.545029,29,0.142021,1.185367,1,0.998548,4.937500,-0.000336,0.199707,0.334119,29,0.089166,1.096343,0,0.000141,5.740224


In [128]:
data_df=data.drop(columns=['index','genre','pacf_rel_rate','if_peroid','entropy','pacf_rel_rate.1','entropy.1','if_peroid.1'])#'entropy','pvalue','if_peroid'
class_to_idx = {x:i for i,x in enumerate(data.genre.unique())}
data_df['y'] = data['genre'].map(class_to_idx)#这两行是将genre中的字符串类别转化为有序数字，保存在y这一列中
data_df.head()

,coefficient,R2,pacf_max,pacf_index,pacf_abs_rate,pvalue,coefficient.1,R2.1,pacf_max.1,pacf_index.1,pacf_abs_rate.1,pvalue.1,y
0,-0.003153,0.841198,4.143352,20,0.165371,0.345547,-0.005579,0.914857,0.433067,26,0.135172,0.079487,0
1,-0.002307,0.881956,7.572991,20,0.241182,0.537526,-0.002212,0.709225,4.376909,29,0.457470,0.000417,0
2,0.002948,0.833641,1.818423,28,0.246616,0.088967,0.001939,0.527923,17.170336,28,0.745046,0.479829,0
3,0.000424,0.611978,33.036178,29,0.875388,0.022661,-0.000481,0.144355,0.307684,5,0.104818,0.000016,0
4,-0.002731,0.775485,0.545029,29,0.142021,0.998548,-0.000336,0.199707,0.334119,29,0.089166,0.000141,0


In [181]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data_df, test_size=0.2,stratify = data_df['y'], random_state=10)
x_train = train_df.drop(columns=['y'])
y_train = train_df['y']
x_test = test_df.drop(columns=['y'])
y_test= test_df['y']

'''
std=StandardScaler().fit(x_train)#生成规则
train_std=std.fit_transform(x_train)#将规则应用与训练集
test_std=std.fit_transform(x_test)#将规则应用于测试集
'''
std=StandardScaler()
std.fit(x_train)#生成规则
train_std=std.transform(x_train)#将规则应用与训练集
test_std=std.transform(x_test)#将规则应用于测试集

In [182]:
test_std

array([[-0.09788595, -1.51284354, -0.14009495, ...,  0.84924795,
        -0.54454199, -0.83327642],
       [ 0.37203418,  0.63770319, -0.13939047, ...,  0.69818591,
         1.37933122,  2.74759832],
       [ 0.94800811,  0.72336878, -0.11701786, ...,  0.84924795,
        -0.76680651, -0.61508707],
       ...,
       [-0.93904404,  0.61311285, -0.07785578, ..., -2.32305494,
        -0.72522694, -0.85023596],
       [ 0.9252328 ,  0.37196786,  0.24019089, ..., -1.56774473,
        -0.91493668,  2.71642051],
       [-0.37309543,  1.14487001,  0.11874495, ..., -1.41668268,
        -0.65438131, -0.82078652]])

In [130]:
#五种常用的分类器模型
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report,confusion_matrix

In [183]:
#默认设置下的各分类器在训练集上分类的表现
models = {'Logistic Regression':LogisticRegression,'Random Forest':RandomForestClassifier,'KNN':KNeighborsClassifier,'Support Vector':SVC,'Naive bayes bernoulli':BernoulliNB}
for i in models:
    clf = models[i]()
    clf.fit(train_std,y_train)
    print(i)
    print(classification_report(y_train,clf.predict(train_std)))

Logistic Regression
              precision    recall  f1-score   support

           0       0.27      0.25      0.26        80
           1       0.25      0.44      0.32        93
           2       0.23      0.29      0.25        84
           3       0.22      0.19      0.20        73
           4       0.33      0.09      0.14        57
           5       0.20      0.25      0.22        84
           6       0.18      0.13      0.15        62
           7       0.33      0.13      0.19        62

    accuracy                           0.24       595
   macro avg       0.25      0.22      0.22       595
weighted avg       0.25      0.24      0.23       595

Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        80
           1       1.00      1.00      1.00        93
           2       1.00      1.00      1.00        84
           3       1.00      1.00      1.00        73
           4       1.00      1.00      1.00 

In [201]:
#SVC
classifier = SVC(kernel="rbf",gamma=1,C=0.7)
classifier = classifier.fit(train_std, y_train)
print("The accuracy of the classifier on the train set is ", (classifier.score(train_std, y_train)))
print("The accuracy of the classifier on the validation set is ", (classifier.score(test_std, y_test)))

The accuracy of the classifier on the train set is  0.9613445378151261
The accuracy of the classifier on the validation set is  0.16778523489932887


In [202]:
train_pred = classifier.predict(train_std)
train_cm = confusion_matrix(y_train, train_pred)
print('训练集的混淆矩阵：')
print(train_cm)

训练集的混淆矩阵：
[[78  2  0  0  0  0  0  0]
 [ 0 93  0  0  0  0  0  0]
 [ 0  0 84  0  0  0  0  0]
 [ 0  2  1 70  0  0  0  0]
 [ 0  3  1  0 52  1  0  0]
 [ 0  0  0  0  0 84  0  0]
 [ 0  6  0  0  0  1 55  0]
 [ 1  4  0  0  0  1  0 56]]


In [200]:
test_pred = classifier.predict(test_std)
train_cm = confusion_matrix(y_test, test_pred)
print('测试集的混淆矩阵：')
print(train_cm)

测试集的混淆矩阵：
[[ 0 20  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0]
 [ 0 19  0  0  0  2  0  0]
 [ 0 15  1  0  0  0  0  0]
 [ 0 15  0  0  0  0  0  0]]


In [177]:
#KNN 默认K=5
classifier=KNeighborsClassifier(7)
classifier=classifier.fit(x_train,y_train)
print("The accuracy of the classifier on the train set is ", (classifier.score(train_std, y_train)))
print("The accuracy of the classifier on the validation set is ", (classifier.score(test_std, y_test)))

The accuracy of the classifier on the train set is  0.1226890756302521
The accuracy of the classifier on the validation set is  0.12751677852348994
